![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the Configuring Egeria Servers Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information about data and technology.  This information is called metadata.

In this hands-on lab you will learn how to configure the metadata servers used by [Coco Pharmaceuticals](https://opengovernance.odpi.org/coco-pharmaceuticals/).

## The scenario

<img src="https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png" style="float:left">

Coco Pharmaceuticals is going through a major business transformation that requires them to drastically reduce their cycle times, collaborate laterally across the different parts of the business and react quickly to the changing needs of their customers. (See [this link](https://opengovernance.odpi.org/coco-pharmaceuticals/) for the background to this transformation).

Part of the changes needed to the IT systems that support the business is the roll out of a distributed metadata and governance capability that is provided by ODPi Egeria.

[Gary Geeke](https://opengovernance.odpi.org/coco-pharmaceuticals/personas/gary-geeke.html) is the IT Infrastructure leader at Coco Pharmaceuticals.

In this hands-on lab Gary is configuring the metadata servers ready for deployment.  Gary's userId is `garygeeke`.

In [ ]:
import requests

adminUserId     = "garygeeke"

He needs to define the metadata servers for Coco Pharmaceuticals.

In [ ]:
organizationName = "Coco Pharmaceuticals"

## Metadata management landscape

Gary has decided to deploy a separate metadata server for each part of the organization that owns assets.
You can think of a metadata server supporting a community of users.  The metadata repositories are as follows:

* cocoMDS1 - Data Lake Operations - used to manage the data in the data lake.
* cocoMDS2 - Governance - used by all of the governance teams to operate the governance programs.
* cocoMDS3 - Research - used by the research teams who are developing new treatments.
* cocoMDS4 - Data Lake Users - used by general business users and the executive team to access data from the data lake.
* cocoMDS5 - Business Systems - used to record information about the operational business systems such as procurements, sales, human resources and finance.
* cocoMDS6 - Manufacturing - used by the warehouse, manufacturing and distribution teams.
* cocoMDSx - Development - used by the development teams building new IT capablity.
* cocoEDGE*i* - Manufacturing sensors edge node servers (many of them).

These metadata servers will each be configured in later parts of this hands-on lab, but first there are decisons to be made about the platform that the servers will run on and how they will be connected together.

### Open Metadata and Governance (OMAG) Server Platforms

The Coco Pharmaceuticals' metadata servers along with the governance servers that interact with them must be hosted on at least one OMAG Server Platform.

The OMAG Server Platform is a single executable (application) that can be started from the command line or a script or as part of a pre-built container environment such as `docker-compose` or `kubernetes`.

If you are running this notebook as part of an Egeria hands on lab then the server platforms you need are already started.  Run the following command to check that the admin platform is running.


In [ ]:
%run common/environment-check.ipynb

----
If one of the platforms is not running, follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).  Once the platforms are running you are ready to proceed.

----
Most of the metadata servers are pretty stable and can share an OMAG Server Platform.
The data lake however requires a lot of active governance so Gary chooses to put all of the metadata and governance servers for the data lake on to their own platform.  The development team requested that their infrastructure is completely separate from the operational systems,
so they are given their own server platform.  Finally each of the edge servers will run an OMAG Server Platform to support
their metadata server.

Figure 1 shows which metadata servers will sit in each platform.

![Figure 1](images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms


The sensor edge node servers used to monitor the warehouse operation and manufacturing process each have their own platform and are not yet included in this notebook.

### Open Metadata Repository Cohorts

Metadata servers communicate via open metadata repository cohorts.  A server can become a member of none, one or many cohorts.  Once it has joined a cohort it can exchange metadata with the other members of that cohort.  So the cohorts define scopes of sharing.

Gary decides to begin with three open metadata repository cohorts:

* **cocoCohort** - The production cohort contains all of the metadata servers that are used to run, coordinate and govern the business.
* **devCohort** - The development cohort where the development teams are building and testing new capablity.  Much of their metadata describes the software components under construction and the governance of the software development lifecycle.
* **iotCohort** - The IoT cohort used to manage the sensors and robots in the manufacturing systems.  The metadata produced by the sensors and robots is only of interest to the manufactuing and governance team.

Figure 2 shows which metadata servers belong to each cohort.

![Figure 2](images/coco-pharmaceuticals-systems-metadata-servers.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

Below are the names of the three cohorts.

In [ ]:
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

At the heart of each cohort is an event topic.  By default, ODPi Egeria uses [Apache Kafka](https://kafka.apache.org/) topics.
The metadata servers will need to be configured with the host name and port where Kafka is running.
The command below pulls the value from an environment variable called `eventBusURLroot` with a default value of
`localhost:9092`.  It is used in all of the server configuration documents to connect it to Kafka.

In [ ]:
eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')

jsonContentHeader = {'content-type':'application/json'}
eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}

## Access services

[The Open Metadata Access Services (OMAS)](https://egeria.odpi.org/open-metadata-implementation/access-services/) provide domain-specific services for data tools, engines and platforms to integrate with open metadata.

The table below shows which access services are needed by each server.


| Access Service       | cocoMDS1 | cocoMDS2 | cocoMDS3 | cocoMDS4 | cocoMDS5 | cocoMDS6 | cocoMDSx | cocoEDGE*i* |
| :------------------- | :------: | :------: | :------: | :------: | :------: | :------: | :------: | :---------: |
| asset-catalog        |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| asset-consumer       |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| asset-owner          |   Yes    |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| community-profile    |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| glossary-view        |   Yes    |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| data-science         |   No     |   No     |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| subject-area         |   No     |   Yes    |   Yes    |   Yes    |   No     |   Yes    |   Yes    |     No      |
| -------------------  |  ------  |  ------  | ------   |  ------  |  ------  |  ------  |  ------  | ----------  |
| governance-program   |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| data-privacy         |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| security-officer     |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| asset-lineage        |   No     |   Yes    |   No     |   No     |   No     |   No     |   No     |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| discovery-engine     |   Yes    |   No     |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| stewardship-action   |   Yes    |   No     |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| data-engine          |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     Yes     |
| data-manager        |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     Yes     |
| governance-engine    |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     No      |
| information-view     |   Yes    |   No     |   No     |   No     |   No     |   Yes    |   No     |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| it-infrastructure    |   No     |   Yes    |   No     |   No     |   No     |   Yes    |   Yes    |     No      |
| project-management   |   No     |   Yes    |   Yes    |   No     |   No     |   Yes    |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |
| software-developer   |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| devops               |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| digital-architecture |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| design-model         |   No     |   No     |   No     |   No     |   No     |   No     |   Yes    |     No      |
| -------------------- |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  |  ------  | ----------  |



## ODPi Egeria Server Configuration Overview

Open metadata servers are configured using REST API calls to an OMAG Server Platform.  Each call either defines a default value or configures a service that must run within the server when it is started.

As each configuration call is made, the OMAG Server Platform builds up a [configuration document](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/configuration-document.html) with the values passed.  When the configuration is finished, the configuration document will have all of the information needed to start the server.

The configuration document will then be deployed with the OMAG Server Platform that is to host the server.  When a request is made to this OMAG Server Platform to start the server, it reads the configuration document and initializes the server with the appropriate services.

## Configuration Set Up

A server can be configured by any OMAG Server Platform - it does not have to be the same platform where the server will run.  For this hands on lab we will use the development team's OMAG Server Platform to create the servers' configuration documents and then deploy them to the platforms where they will run.

In [ ]:
adminPlatformURL = devPlatformURL

The URLs for the configuration REST APIs have a common structure and begin with the following root:

In [ ]:
adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

Many of Coco Pharmaceuticals' metadata servers need a local repository to store metadata about the data and processing occuring in the data lake.

Egeria includes two types of repositories natively. One is an **in-memory repository** that stores metadata in hash maps. It is useful for demos and testing because a restart of the server results in an empty metadata repository. However, if you need metadata to persist from one run of the server to the next, you should use the **local graph repository**.

The choice of local repository is made by specifying the local repository mode. The variables below show the two options. The `metadataRepositoryType` identfies which one is going to be used in the configuration.

In [ ]:
inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

metadataRepositoryType   = inMemoryRepositoryOption

Egeria supports instance based security.  These checks can be customized through an **Open Metadata Security Verifier**. Coco Pharaceuticals have written their own connector to support the specific rules of their industry.  The Connection definition below tells a server how to load this connector.  It needs to be included in each metadata server's configuration document.

In [ ]:
serverSecurityConnectionBody = {
    "class": "Connection",
    "connectorType": {
        "class": "ConnectorType",
        "connectorProviderClassName": "org.odpi.openmetadata.metadatasecurity.samples.CocoPharmaServerSecurityProvider"
    }
}

Finally, to ensure that a caller can not request too much metadata in a single request, it is possible to set a maximum page size for requests that return a list of items.  The maximum page size puts a limit on the number of items that can be requested.  The variable below defines the value that will be added to the configuration document for each server.

In [ ]:
maxPageSize = '100'

## Configuring cocoMDS1 - Data Lake Operations metadata server

This section configures the `cocoMDS1` server.  The server name is passed on every configuration call to identify which configuration document to update with the new configuration.  The configuration document is created automatically on first use.

In [ ]:
mdrServerName          = "cocoMDS1"
mdrServerUserId        = "cocoMDS1npa"
mdrServerPassword      = "cocoMDS1passw0rd"
mdrServerPlatform      = dataLakePlatformURL
metadataCollectionName = "Data Lake Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": ["quarantine", "clinical-trials", "research", "data-lake", "trash-can"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)

accessServiceOptions["DefaultZones"] = [ "quarantine" ]
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'stewardship-action', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'information-view', accessServiceOptions)

print("\nDone.")

----

## Configuring cocoMDS2 - Governance metadata server

This section configures the `cocoMDS2` server.  This server is configured in a similar way to cocoMDS1 except that is has different Open Metadata Access Services (OMASs) enabled and it joins all of the cohorts.

The code below covers the basic set up of the server properties, security, event bus and local repository.

In [ ]:
mdrServerName          = "cocoMDS2"
mdrServerUserId        = "cocoMDS2npa"
mdrServerPassword      = "cocoMDS2passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Governance Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
# Note: cohort membership is configured for all of the cohorts here
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, iotCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-program', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-privacy', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'security-officer', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-lineage', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'it-infrastructure', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', {})

print("\nDone.")

----

## Configuring cocoMDS3 - Research

Server cocoMDS3 is used by the research teams who are developing new treatments.
These teams are working with their own assets as well as assets coming from the data lake.
They have their own discovery server to automate analysis of their data.
They are also creating new data science models that they run by hand, or deploy to the discovery server.

In [ ]:
mdrServerName          = "cocoMDS3"
mdrServerUserId        = "cocoMDS3npa"
mdrServerPassword      = "cocoMDS3passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Research Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": ["personal-files", "clinical-trials", "research", "data-lake", "trash-can"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)

accessServiceOptions["DefaultZones"] = [ "personal-files" ]
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'stewardship-action', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', accessServiceOptions)

print("\nDone.")

----
## Configuring cocoMDS4 - Data Lake Users

Server cocoMDS4 used by general business users and the executive team to access data from the data lake.
It does not have a repository of its own.  Instead it issues federated queries to the other repositories in the `cocoCohort`.

In [ ]:
mdrServerName          = "cocoMDS4"
mdrServerUserId        = "cocoMDS4npa"
mdrServerPassword      = "cocoMDS4passw0rd"
mdrServerPlatform      = dataLakePlatformURL
metadataCollectionName = "Data Lake Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
# Note: no metadata repository or collection configuration here
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": [ "data-lake" ]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})

print("\nDone.")

----

## Configuring cocoMDS5 - Business Systems

Server cocoMDS5 is a repository proxy to an ETL tool called `iisCore01`.  This ETL tool is well established in Coco Pharmaceuticals and has a built-in metadata repository that contains information about their operational business systems such as procurement, sales, human resources and finance.

This ETL tool has its own user interface and services so the OMASs are not enabled.

In [ ]:
mdrServerName          = "cocoMDS5"
mdrServerUserId        = "cocoMDS5npa"
mdrServerPassword      = "cocoMDS5passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Business Systems Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureRepositoryProxyDetails(adminPlatformURL, adminUserId, mdrServerName, "org.odpi.openmetadata.adapters.repositoryservices.readonly.repositoryconnector.ReadOnlyOMRSRepositoryConnectorProvider")
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)
# Note: no access service configuration here
# Still need to add startup Archive 

print("\nDone.")

----

## Configuring cocoMDS6 - Manufacturing

Server cocoMDS6 is the repository server used by the warehouse, manufacturing and distribution teams.  It supports the systems for this part of the organization and acts as a hub for monitoring the IoT environment.

In [ ]:
mdrServerName          = "cocoMDS6"
mdrServerUserId        = "cocoMDS6npa"
mdrServerPassword      = "cocoMDS6passw0rd"
mdrServerPlatform      = corePlatformURL
metadataCollectionName = "Manufacturing Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, cocoCohort)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, iotCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": [ "manufacturing" ],
    "DefaultZones" : [ "manufacturing"]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'stewardship-action', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-manager', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'governance-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'information-view', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'it-infrastructure', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', accessServiceOptions)

print("\nDone.")

----

## Configuring cocoMDSx - Development

Server cocoMDSx is used by the development teams building new IT capablity.  It will hold all of the software component assets and servers used for development and devOps.  The development teams have their own OMAG Server Platform and cohort called 'devCohort'.

In [ ]:
mdrServerName          = "cocoMDSx"
mdrServerUserId        = "cocoMDSxnpa"
mdrServerPassword      = "cocoMDSxpassw0rd"
mdrServerPlatform      = devPlatformURL
metadataCollectionName = "Development Catalog"

print("Configuring " + mdrServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, mdrServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, mdrServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, mdrServerName, mdrServerUserId)
configurePassword(adminPlatformURL, adminUserId, mdrServerName, mdrServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, mdrServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)
configureDescriptiveName(adminPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)

print("\nConfiguring " + mdrServerName + " Access Services (OMAS)...")

accessServiceOptions = {
    "SupportedZones": [ "sdlc" ],
    "DefaultZones": [ "sdlc" ]
}
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-catalog', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-consumer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'asset-owner', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'community-profile', {"KarmaPointPlateau":"500"})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'glossary-view', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'data-science', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'subject-area', {})
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'discovery-engine', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'stewardship-action', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'it-infrastructure', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'project-management', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'software-developer', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'devops', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'digital-architecture', accessServiceOptions)
configureAccessService(adminPlatformURL, adminUserId, mdrServerName, 'design-model', accessServiceOptions)

print("\nDone.")

----
## Configuring the Discovery Server

The discovery server is a special kind of governance server that hosts one or more discovery engines.  A discovery engine is a set of discovery services that analyzes the content of an organization's assets.  For example, a discovery service may open up a data set and assess the quality of the data inside.

The result of a discovery service's analysis is stored in a metadata server as a discovery analysis report that is chained off of the asset's definition.  This report can be retrieved either through the discovery server's API or through the metadata server's APIs, specifically the Discovery Engine OMAS and the Asset Owner OMAS.

The behavior of the discovery server, its discovery engines and the discovery services within are defined in the [Open Discovery Framework (ODF)](https://egeria.odpi.org/open-metadata-implementation/frameworks/open-discovery-framework/).  This framework enables new implementations of discovery services to be deployed to the discovery engines.

Typically a discovery server is deployed close to where the data is stored because it can generate a lot of network
traffic when it is retrieving all of the content of an asset.

Coco Pharmaceuticals runs one discovery server for its data lake.  It is called `findItDL01` and it runs on the This discovery server has three discovery engines running on it:

* **AssetDiscovery** - extracts metadata about different types of assets on request.
* **AssetDeduplicator** - detects and reports on asset definitions in the open metadata repositories that seem to be
  duplicate descriptions of the same physical asset.
* **AssetQuality** - assesses the quality of the content of assets on request.

The commands below configure the discovery server.  The definitions of the discovery engines and their services
are retrieved from the `cocoMDS1` metadata server through its Discovery Engine OMAS.


In [ ]:
discoServerName        = "findItDL01"
discoServerPlatform    = dataLakePlatformURL
discoServerUserId      = "findItDL01npa"
discoServerPassword    = "findItDL01passw0rd"

mdrServerName          = "cocoMDS1"
mdrServerPlatform      = dataLakePlatformURL

print("Configuring " + discoServerName + "...")

configurePlatformURL(adminPlatformURL, adminUserId, discoServerName, discoServerPlatform)
configureMaxPageSize(adminPlatformURL, adminUserId, discoServerName, maxPageSize)
clearServerType(adminPlatformURL, adminUserId, mdrServerName)
configureOwningOrganization(adminPlatformURL, adminUserId, discoServerName, organizationName)
configureUserId(adminPlatformURL, adminUserId, discoServerName, discoServerUserId)
configurePassword(adminPlatformURL, adminUserId, discoServerName, discoServerPassword)
configureSecurityConnection(adminPlatformURL, adminUserId, discoServerName, serverSecurityConnectionBody)
configureEventBus(adminPlatformURL, adminUserId, discoServerName, eventBusBody)
configureDefaultAuditLog(adminPlatformURL, adminUserId, discoServerName)

print("\nConfiguring " + discoServerName + " as a discovery server ...")

discoveryEngines = ["AssetDiscovery", "AssetDeduplicator", "AssetQuality"]

configureDiscoveryEngineServices(adminPlatformURL, adminUserId, discoServerName, mdrServerName, mdrServerPlatform, discoveryEngines)

----

# Deploying server configuration

The commands that have been issued so far have created a configuration document for each server.
These configuration documents are currently local to the Development OMAG Server Platform where the
adminstration commands were issued (figure 3).

![Figure 3](images/creating-configuration-documents.png)
> **Figure 3:** Creating configuration documents using administration commands

If servers are to be started on the other server platforms then their configuration documents
need to be deployed (copied) to these platforms (figure 4).

![Figure 4](images/deploying-configuration-documents.png)
> **Figure 4:** Deploying configuration documents

However, before deploying the configuration documents, the receiving OMAG Server Platforms
need to be running.

The code below checks the Core and Data Lake OMAG Server Platforms are running.

In [ ]:
print("\nChecking OMAG Server Platform availability...")

checkServerPlatform("Data Lake Platform", dataLakePlatformURL)
checkServerPlatform("Core Platform", corePlatformURL)
checkServerPlatform("Dev Platform", devPlatformURL)

print ("\nDone.")

----
Make sure the each of the platforms is running.

----
The commands below deploy the server configuration documents to the server platforms where the
servers will run.

In [ ]:


print("\nDeploying server configuration documents to appropriate platforms...")
    
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS1", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS2", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS3", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS4", dataLakePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS5", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDS6", corePlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "cocoMDSx", devPlatformURL)
deployServerToPlatform(adminPlatformURL, adminUserId, "findItDL01", dataLakePlatformURL)

print("\nDone.")

----